In [1]:
import os
import sys
import json
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
w_dir

'/work/nlu_IR'

In [2]:
sys.path.append(w_dir+'/nlu_IR')

In [3]:
from nlu_IR import config
from nlu_IR import std

In [4]:
config.COS_DEV

PosixPath('/work/nlu_IR/data/cosQA/movieqa_1.0/movieqa_dev.json')

In [5]:
data = std.json_load(config.COS_DEV)

In [6]:
data[2463]

{'title': {'en': 'Alice Adams (1935 film)',
  'zh': None,
  'cn': None,
  'tw': None},
 'ref': {'en': {'wikibase': 'Q1619523',
   'pageid': 7301880,
   'url': 'https://en.wikipedia.org/wiki/Alice_Adams_(1935_film)',
   'wikidata_url': 'https://www.wikidata.org/wiki/Q1619523'},
  'zh': {'wikibase': None, 'pageid': None, 'url': None, 'wikidata_url': None}},
 'infobox': {'en': [{'name': 'Alice Adams'},
   {'image': 'Alice-Adams-Poster-1935.jpg'},
   {'caption': 'Theatrical release poster'},
   {'director': '[[George Stevens]]'},
   {'producer': '[[Pandro S. Berman]]'},
   {'based on': "{{based on|''[[Alice Adams (novel)|Alice Adams]]''<br>1921 novel|[[Booth Tarkington]]}}"},
   {'screenplay': '[[Dorothy Yost]]<br>[[Mortimer Offner]]<br>[[Jane Murfin]]'},
   {'starring': '[[Katharine Hepburn]]<br>[[Fred MacMurray]]<br>[[Fred Stone]]<br>[[Evelyn Venable]]'},
   {'music': '[[Max Steiner]]<br>[[Roy Webb]]'},
   {'cinematography': '[[Robert De Grasse]]'},
   {'editing': '[[Jane Loring]]'},
   

In [7]:
data[0]['QUESTIONS'][0]['QTEXT_CN']

'特穆拉·莫里森演什么电影？'

In [8]:
from nlu_IR.lucene_search.cosQA_search import CosQASearcher
from nlu_IR.evaluation.eval import eval_sp

In [9]:
test_searcher = CosQASearcher('en')

In [10]:
test_searcher.search('what movies did Temuera Morrison act in?', 10)

[('Q3536311',
  'Tracker (2011 film)',
  'Tracker is a 2011 British–New Zealand action-thriller film directed by Ian Sharp and starring Ray Winstone and Temuera Morrison. It is set in 1903 New Zealand.',
  9.362552642822266),
 ('Q5243353',
  'Once Were Warriors',
  "Once Were Warriors is New Zealand author Alan Duff's bestselling first novel, published in 1990. It tells the story of an urban Māori family, the Hekes, and portrays the reality of domestic violence in New Zealand. It was the basis of a 1994 film of the same title, directed by Lee Tamahori and starring Rena Owen and Temuera Morrison, which made its U.S. premiere at the Hawaii International Film Festival. The novel was followed by two sequels, What Becomes of the Broken Hearted? (1996) and Jake's Long Shadow (2002).",
  9.084122657775879),
 ('Q2155524',
  'River Queen',
  'River Queen is a 2005 New Zealand-British war drama film directed by  Vincent Ward and starring Samantha Morton, Kiefer Sutherland, Cliff Curtis, Temuera 

In [8]:
mySearcher = CosQASearcher('zh')

In [13]:
mySearcher.search('特穆拉·莫里森演什么电影？', 5)

[('Q1184001',
  'What a Girl Wants',
  '女孩想要什么可以指：“女孩想要什么”（克里斯蒂娜·阿奎莱拉·宋），1999女孩想要什么（电影），2003年阿曼达·拜恩斯主演的电影“女孩想要什么”（B2K歌曲），2003年“女孩想要什么”，一首4分钟前为穆齐克创作的歌曲',
  7.66987943649292),
 ('Q2155524',
  'River Queen',
  '河皇后是2005年由文森特·沃德执导，萨曼莎·莫顿、基弗·萨瑟兰、克里夫·柯蒂斯、特穆拉·莫里森和斯蒂芬·雷亚主演的新西兰-英国战争戏剧电影。这部电影的评价参差不齐，但在新西兰的票房表现良好。',
  7.614450454711914),
 ('Q1184001',
  'What a Girl Wants',
  '女孩想要什么可以指：“女孩想要什么”（克里斯蒂娜·阿奎莱拉之歌），1999女孩想要什么（电影），由阿曼达·拜恩斯主演的2003年电影“女孩想要什么”（B2K之歌），2003年“女孩想要什么”，一首4分钟前为穆齐克创作的歌',
  7.595847129821777),
 ('Q529851',
  'Temuera Morrison',
  '特穆拉·德里克·莫里森（1960年12月26日出生）是新西兰演员，他在新西兰肥皂剧《肖特兰街》（Shortland Street）中扮演的Hone Ropata博士首次获得认可。他在1994年的电影《曾经是勇士》和1999年的续集《伤心的人是什么》中饰演杰克·海克，获得了评论界的好评？。他在《星球大战：第二集克隆人的进攻》（2002年）和《星球大战：第二集克隆人的进攻》和《星球大战：第三集西斯的复仇》（2005年）中扮演了詹戈·费特（Jango Fett），从而在国际上享有盛名。他还因在《莫阿纳》（2016）中饰演片名人物之父的崔局长以及在《阿奎曼》（2018）中饰演亚瑟·柯里的父亲托马斯而闻名。',
  7.592428684234619),
 ('Q3536311',
  'Tracker (2011 film)',
  '跟踪器是一部2011年英国-新西兰动作惊悚片，由伊恩·夏普执导，雷·温斯顿和特穆拉·莫里森主演。它设定在1903年的新西兰。',
  7.053170680999756)]

In [25]:
def inspect_cosqa(data):
    
    num_gold_sp = 0
    valid_data = []
    invalid_data = []
    for d in tqdm(data):
        try:
            assert len(d['QUESTIONS']) == 1, 'There are {} questions!'.format(len(d['QUESTIONS']))
            gold_sp = d['QUESTIONS'][0]['SHINT_']
            num_gold_sp += len(gold_sp)
            valid_data.append(d)
        except Exception:
            
            invalid_data
    print("average_gold_sp:{}".format(num_gold_sp/len(data)))

In [26]:
inspect_cosqa(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


KeyError: 'QUESTIONS'

In [15]:
def search_batch(q_texts, searcher, ton_n):
    predictions = []
    
    for q_text in enumerate(q_texts):
        prediction = searcher.search(q_text, top_n)
        predictions.append([p[0] for p in prediction])
    return predictions

In [ ]:
def eval_cosqa(data):
    searcher_cn = CosQASearcher('zh')
    searcher_en = CosQASearcher('en')
    
    cn_q_texts = []
    en_q_texts = []
    golds = []
    for d in data:
        assert len(d['QUESTIONS']) == 1, 'There are {} questions!'.format(len(d['QUESTIONS']))
        cn_q_texts.append(['QUESTIONS'][0]['QTEXT_CN'])
        en_q_texts.append(['QUESTIONS'][0]['QTEXT'])
        golds.append(['QUESTIONS'][0]['SHINT_'])
    
    #search
    search_batch(cn_q_texts, searcher_cn, 10)
        

In [9]:
test_query = '特穆拉·莫里森演什麼電影？'

In [10]:
mySearcher.search(test_query, 5)

[('Q454328',
  'Stacey Dash',
  '斯泰西·劳雷塔·达什（1966年或1967年1月20日出生）是一位美国女演员和前脱口秀主持人。达什在1995年的故事片《无影无踪》和同名电视连续剧中扮演迪翁·玛丽·达文波特。她还出演过电影《感动》、《莫钱》、《文艺复兴的男人》和《从高处看》。达什的其他电视作品包括在《犯罪现场调查：单身女性》和真人秀《名人马戏团》中的露面。她还出现在卡尔·托马斯的音乐录影带《情感》和坎耶·韦斯特的《所有的堕落》中。',
  8.23513412475586),
 ('Q2155524',
  'River Queen',
  '河皇后是2005年由文森特·沃德执导，萨曼莎·莫顿、基弗·萨瑟兰、克里夫·柯蒂斯、特穆拉·莫里森和斯蒂芬·雷亚主演的新西兰-英国战争戏剧电影。这部电影的评价参差不齐，但在新西兰的票房表现良好。',
  7.406106948852539),
 ('Q1129503',
  'Glen or Glenda',
  '《格伦或格伦达》是一部1953年的美国剥削电影，由埃德·伍德执导并主演（在他主演的《丹尼尔·戴维斯》中有人称赞），影片中贝拉·卢戈西和伍德当时的女友多洛丽丝·富勒是主角。该片由乔治·韦斯（George Weiss）制作，同年他还制作了一部剥削电影《试管婴儿》（Test Tube Babies），是一部关于变装和变性的纪录片，性质上是半自传体的。伍德本人就是一个变装者，这部电影是对宽容的恳求。它被广泛认为是有史以来最差的电影之一。然而，由于其低成本的生产价值和独特的风格，它已经成为一个邪典電影。',
  7.125361919403076),
 ('Q3536311',
  'Tracker (2011 film)',
  '跟踪器是一部2011年英国-新西兰动作惊悚片，由伊恩·夏普执导，雷·温斯顿和特穆拉·莫里森主演。它设定在1903年的新西兰。',
  7.053170680999756),
 ('Q7136855',
  'Paresh Mokashi',
  '帕雷什莫卡希（1969年2月6日出生）是一名印度电影制片人、制片人、演员和戏剧导演；主要在马拉地电影院和马拉地剧院工作。他开始为剧院做后台工作，除了拍电影外，很少为戏剧做次要角色。1999年，莫卡希首次执导马拉地戏剧《桑吉